## 1. 군집분석

### 데이터 설명
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.     
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.     
- Description: Product (item) name. Nominal.    
- Quantity: The quantities of each product (item) per transaction. Numeric.     
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.   
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.    
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.     
- Country: Country name. Nominal, the name of the country where each customer resides.     


### 1.1 F(소비자별 구매빈도), M(소비자별 총 구매액) feature를 새로 생성해서 그 결과값으로 탐색적 분석 실시


### 1.2 F, M feature 기반으로 군집분석 실시, 필요시 이상값 보정

### 1.3 군집 결과의 적합성을 군집 내 응집도, 군집 간 분리도의 개념을 사용해서 서술

### 1.4 적합된 군집 별 특성에 대한 의견과 비즈니스적 판단 제시

## 2번 시계열분석
- 데이터 설명    
  각 raw는 관광지 A의 1990년 1월 부터 25년동안의 매달 평균 이용객 숫자이다.

### 2-1 EDA와 시각화를 진행하라

### 2-2 결측치 처리와 해당 결측치 처리 방식에 대한 논리적 근거를 제시하라

### 2-3 계절성을 반영한 시계열 모델을 제시하고 정확도 측면에서 모델 성능 평가 할 것

### 2-4 분석 결과 활용 가능 여부에 대한 분석 전문가로서의 제안

## 3번
### 3-1 서울에서 영동까지 100km/h로 가고 영동에서 서울까지 80km/로 돌아왔을 때, 평균 속도는?

### 3-2 연매출이 3000, 4000, 5000이었다면 연평균 몇배가 증가한 것인가?

### 3-3 남성, 여성의 등산, 수영에 대한 취미 선호도 빈도표(2x2)를 보고, 남성 중에서 등산을 좋아할 확률을 구하시오

![image](https://github.com/Datamanim/datarepo/blob/main/adp/25/problem3_3.png?raw=true)

### 3-4 표본 10개의 분산이 90일 때 신뢰도 95%로 모분산의 신뢰구간을 추정

## 4번 임상 대상 20명에 대해 혈압약 투약 이전과 이후의 차이가 24, 표준편차 9  신뢰구간 95%, 차이가 존재하는지 확인하려한다.


### 4-1 귀무가설과 연구가설을 제시하시오

### 4-2 검정 후 귀무가설 기각 여부 제시

## 5번 공장 X,Y,Z의 평균 출하 소요시간을 여러 일자에 거쳐 측정한 데이터이다. 각 공장의 중위수의 차이가 존재하는지 확인하려 한다.
- problem5.csv

### 5-1. 연구가설과 귀무가설을 설정하시오

### 5-2. 검정통계량을 구하고 가설을 채택하시오

## 6번 1개년 50억원, 2개년 60억원, 3개년 70억원의 예산을 가지고 NPV(순현재가치)가 가장 높아지는 안을 제시하시오.

![image](https://github.com/Datamanim/datarepo/blob/main/adp/25/problem6.png?raw=true)